In [1]:
import pandas as pd
import numpy as np
import cv2
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout
from keras.optimizers import SGD
from keras.utils import to_categorical
import os
from sklearn.model_selection import train_test_split
from tensorflow.examples.tutorials.mnist import input_data
from  tqdm import tqdm


Using TensorFlow backend.


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
#mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

#_train, y_train = mnist.train.images,mnist.train.labels
#x_test, y_test = mnist.test.images, mnist.test.labels

In [3]:
#x_train.shape

In [4]:
#y_test.shape

In [5]:
DST_SIZE = (227, 227, 3)
CLASS_NUM = 43
ACT_FUNC = 'relu'#'sigmoid'#'relu'#'tanh'

#### 标准的形式 x_trainReshaped.shape (55000, 28, 28, 1) amount height width channel

In [6]:
names = pd.read_csv("unstandard_file.csv")
blacklistSet = names["Filename"]

In [7]:
imgList = []
imageDir = "GTSRB_train_jpgs/"

label = []
for file in tqdm(os.listdir(imageDir)):
    if file not in blacklistSet:
        label.append(int(file.split("#")[0]))
        #mat = cv2.resize(cv2.cvtColor(cv2.imread(imageDir + file), cv2.COLOR_BGR2GRAY), (DST_SIZE[0], DST_SIZE[1]))
        mat = cv2.resize(cv2.imread(imageDir + file), (DST_SIZE[0], DST_SIZE[1]))
        mat = mat.reshape(DST_SIZE)
        imgList.append(mat) 
xset = np.array(imgList)
yset = np.array(label)

100%|██████████| 39209/39209 [00:13<00:00, 2809.02it/s]


In [8]:
print(xset.shape, yset.shape)

(39209, 227, 227, 3) (39209,)


In [9]:
yset_onehot=to_categorical(yset,num_classes=CLASS_NUM)
yset_onehot.shape

(39209, 43)

In [10]:
train_x, test_x, train_y, test_y = train_test_split(xset, yset_onehot, test_size=0.1, random_state=42)

In [13]:
print("train_x", train_x.shape)
print("train_y", train_y.shape)
print("test_x", test_x.shape)
print("test_y", test_y.shape)

train_x (35288, 227, 227, 3)
train_y (35288, 43)
test_x (3921, 227, 227, 3)
test_y (3921, 43)


In [14]:
# create model
model = Sequential()  
model.add(Conv2D(96,(11,11),strides=(4,4),input_shape=DST_SIZE ,padding='valid',activation=ACT_FUNC,kernel_initializer='uniform'))  
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
model.add(Conv2D(256,(5,5),strides=(1,1),padding='same',activation=ACT_FUNC, kernel_initializer='uniform'))  
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
model.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation=ACT_FUNC, kernel_initializer='uniform'))  
model.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation=ACT_FUNC, kernel_initializer='uniform'))  
model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation=ACT_FUNC, kernel_initializer='uniform'))  
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
model.add(Flatten())  
model.add(Dense(4096,activation=ACT_FUNC))  
model.add(Dropout(0.5))  
model.add(Dense(4096,activation=ACT_FUNC))  
model.add(Dropout(0.5))  
model.add(Dense(CLASS_NUM,activation='softmax'))  
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])  
model.summary()  
#事实证明，对于分类问题，使用交叉熵(cross entropy)作为损失函数更好些
model.compile(
    loss='categorical_crossentropy',
    optimizer=SGD(lr=0.1),
    metrics=['accuracy']
)

#train model
model.fit(train_x, train_y,
          batch_size=128,
          epochs=2, 
          verbose = 1,
          validation_data = (test_x, test_y))

#evaluate model

score = model.evaluate(test_x,test_y)
print("Total loss on Testing Set:", score[0])
print("Accuracy of Testing Set:", score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 13, 13, 384)       885120    
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 13, 13, 384)       1327488   
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 13, 13, 256)       884992    
__________